<a href="https://colab.research.google.com/github/sumanthd17/aspect-based-sentiment/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 12.7 GB  | Proc size: 160.9 MB
GPU RAM Free: 15079MB | Used: 0MB | Util   0% | Total 15079MB


In [2]:
!pip install transformers

In [49]:
import time
import datetime
import random
from tqdm import tqdm

import numpy as np
import pandas as pd

import nltk
nltk.download('punkt')

import torch
import torch.nn.functional as F
import transformers as optimus

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
!git clone https://github.com/sumanthd17/aspect-based-sentiment.git

fatal: destination path 'aspect-based-sentiment' already exists and is not an empty directory.


In [5]:
cd aspect-based-sentiment

/content/aspect-based-sentiment


In [6]:
def load_train_data(input_dir):
    df = pd.read_csv(input_dir + "train-QA.csv", sep="\t", names=['id', 'ques', 'ans', 'sentiment'])
    return df

In [7]:
def load_val_data(input_dir):
    df = pd.read_csv(input_dir + "val-QA.csv", sep="\t", names=['id', 'ques', 'ans', 'sentiment'])
    return df

In [53]:
def hyper_params():
    BATCH_SIZE = 32
    MAX_SEQ_LENGTH = 256
    LEARNING_RATE = 2e-5
    EPOCHS = 5
    WARMUP = 0.1
    return BATCH_SIZE, MAX_SEQ_LENGTH, LEARNING_RATE, WARMUP, EPOCHS

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
!python create_data.py

2977
747


In [54]:
train_data = load_train_data('QA_pairs/')
val_data = load_val_data('QA_pairs/')

batch_size, max_seq_len, lr, warmup, epochs = hyper_params()
num_training_steps = int(len(train_data) / batch_size)
num_warmup_steps = warmup * num_training_steps
print(len(train_data))
print(len(val_data))

45025
11246


In [12]:
tokenizer_class, pretrained_weights = (
    optimus.BertTokenizer,
    "bert-base-uncased",
)

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)

In [13]:
sent2idx = {
    'None': 0,
    'Positive': 1,
    'Negative': 2
}

In [30]:
train_data = train_data[:10000]
val_data = val_data[:5000]

In [31]:
train = pd.DataFrame()

question_tokens = train_data['ques'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))
q_segment_tokens = question_tokens.apply(lambda x: [0]*len(x))

answer_tokens = train_data['ans'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))
answer_tokens = answer_tokens.apply(lambda x: [i for i in x if i != 101])
a_segment_tokens = answer_tokens.apply(lambda x: [1]*len(x))

train['input_ids'] = question_tokens + answer_tokens
train['segment_ids'] = q_segment_tokens + a_segment_tokens

train['input_mask'] = ''
for i, row in train.iterrows():
  train.iloc[i]['input_mask'] = [1]*len(train.iloc[i]['input_ids'])

train['input_mask'] = train['input_mask'].apply(lambda x: x + (max_seq_len - len(x))*[0])
train['input_ids'] = train['input_ids'].apply(lambda x: x + (max_seq_len - len(x))*[0])
train['segment_ids'] = train['segment_ids'].apply(lambda x: x + (max_seq_len - len(x))*[0])

train['label'] = ''
train['label'] = train_data['sentiment'].apply(lambda x: sent2idx[x])

In [32]:
val = pd.DataFrame()

question_tokens = val_data['ques'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))
q_segment_tokens = question_tokens.apply(lambda x: [0]*len(x))

answer_tokens = val_data['ans'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))
answer_tokens = answer_tokens.apply(lambda x: [i for i in x if i != 101])
a_segment_tokens = answer_tokens.apply(lambda x: [1]*len(x))

val['input_ids'] = question_tokens + answer_tokens
val['segment_ids'] = q_segment_tokens + a_segment_tokens

val['input_mask'] = ''
for i, row in val.iterrows():
  val.iloc[i]['input_mask'] = [1]*len(val.iloc[i]['input_ids'])

val['input_mask'] = val['input_mask'].apply(lambda x: x + (max_seq_len - len(x))*[0])
val['input_ids'] = val['input_ids'].apply(lambda x: x + (max_seq_len - len(x))*[0])
val['segment_ids'] = val['segment_ids'].apply(lambda x: x + (max_seq_len - len(x))*[0])

val['label'] = ''
val['label'] = train_data['sentiment'].apply(lambda x: sent2idx[x])

In [33]:
train_data['sentiment'].value_counts()

None        9110
Positive     624
Negative     266
Name: sentiment, dtype: int64

In [34]:
val_data['sentiment'].value_counts()

None        4664
Positive     244
Negative      92
Name: sentiment, dtype: int64

In [35]:
train['input_ids'].apply(lambda x: len(x)).unique()

array([256])

In [36]:
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.sampler import RandomSampler

In [37]:
input_ids = torch.tensor([r for r in train['input_ids']], dtype=torch.long)
input_masks = torch.tensor([r for r in train['input_mask']], dtype=torch.long)
label_ids = torch.tensor([r for r in train['label']], dtype=torch.long)
segment_ids = torch.tensor([r for r in train['segment_ids']], dtype=torch.long)

In [38]:
train_dataset = TensorDataset(input_ids, input_masks, segment_ids, label_ids)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

In [39]:
input_ids = torch.tensor([r for r in val['input_ids']], dtype=torch.long)
input_masks = torch.tensor([r for r in val['input_mask']], dtype=torch.long)
label_ids = torch.tensor([r for r in val['label']], dtype=torch.long)
segment_ids = torch.tensor([r for r in val['segment_ids']], dtype=torch.long)

In [40]:
val_dataset = TensorDataset(input_ids, input_masks, segment_ids, label_ids)
val_sampler = RandomSampler(val_dataset)
val_dataloader = DataLoader(val_dataset, sampler=val_sampler, batch_size=batch_size)

In [41]:
model_class, pretrained_weights = (
    optimus.BertForSequenceClassification,
    "bert-base-uncased",
)

model = model_class.from_pretrained(pretrained_weights, num_labels=3)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [42]:
model = model.to(device)

In [43]:
optimizer_params = [
  {'params': [p for n, p in model.named_parameters()], 'weight_decay_rate': 0.0}
]

In [55]:
optimizer = optimus.AdamW(model.parameters(), lr=lr, correct_bias=False)
scheduler = optimus.get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)

In [28]:
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [56]:
training_stats = []

total_t0 = time.time()

for epoch_i in range(0, 1):
    ## TRAINING
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_train_loss = 0

    model.train()

    for step, batch in enumerate(tqdm(train_dataloader)):

        if step % 100 == 0 and not step == 0:
            elapsed = str(datetime.timedelta(seconds=int(round(time.time() - t0))))
            
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        input_ids, input_mask, segment_ids, label_ids = batch
  
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)
        label_ids = label_ids.to(device)

        model.zero_grad()        

        loss, _ = model(input_ids=input_ids,
                        attention_mask=input_mask,
                        token_type_ids=segment_ids,
                        labels=label_ids)

        total_train_loss += loss.item()

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)          
    
    training_time = str(datetime.timedelta(seconds=int(round(time.time() - t0))))

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    ## VALIDATION

    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for step, batch in enumerate(tqdm(val_dataloader)):
        
        input_ids, input_mask, segment_ids, label_ids = batch
  
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)
        label_ids = label_ids.to(device)
        
        with torch.no_grad():        
            loss, logits = model(input_ids=input_ids,
                        attention_mask=input_mask,
                        token_type_ids=segment_ids,
                        labels=label_ids)
            
        total_eval_loss += loss.item()

        logits = F.softmax(logits, dim=-1)
        logits = logits.detach().cpu().numpy()
        label_ids = label_ids.to('cpu').numpy()
        outputs = np.argmax(logits, axis=1)

        total_eval_accuracy += np.sum(outputs == label_ids)
        

    avg_val_accuracy = total_eval_accuracy / len(val_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    avg_val_loss = total_eval_loss / len(val_dataloader)
    
    validation_time = str(datetime.timedelta(seconds=int(round(time.time() - t0))))
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(str(datetime.timedelta(seconds=int(round(time.time() - total_t0))))))





  0%|          | 0/313 [00:00<?, ?it/s]

======== Epoch 1 / 5 ========
Training...






  0%|          | 1/313 [00:01<07:28,  1.44s/it]



  1%|          | 2/313 [00:02<07:23,  1.43s/it]



  1%|          | 3/313 [00:04<07:21,  1.43s/it]



  1%|▏         | 4/313 [00:05<07:20,  1.43s/it]



  2%|▏         | 5/313 [00:07<07:19,  1.43s/it]



  2%|▏         | 6/313 [00:08<07:19,  1.43s/it]



  2%|▏         | 7/313 [00:10<07:19,  1.44s/it]



  3%|▎         | 8/313 [00:11<07:19,  1.44s/it]



  3%|▎         | 9/313 [00:12<07:18,  1.44s/it]



  3%|▎         | 10/313 [00:14<07:17,  1.45s/it]



  4%|▎         | 11/313 [00:15<07:18,  1.45s/it]



  4%|▍         | 12/313 [00:17<07:17,  1.45s/it]



  4%|▍         | 13/313 [00:18<07:16,  1.46s/it]



  4%|▍         | 14/313 [00:20<07:15,  1.46s/it]



  5%|▍         | 15/313 [00:21<07:13,  1.46s/it]



  5%|▌         | 16/313 [00:23<07:11,  1.45s/it]



  5%|▌         | 17/313 [00:24<07:09,  1.45s/it]



  6%|▌         | 18/313 [00:25<07:06,  1.45s/it]



  6%|▌         | 19/313 [00:27<07:04,  1.44s/it]



  6%|▋         | 

  Batch   100  of    313.    Elapsed: 0:02:20.






 32%|███▏      | 101/313 [02:21<04:54,  1.39s/it]



 33%|███▎      | 102/313 [02:23<04:53,  1.39s/it]



 33%|███▎      | 103/313 [02:24<04:51,  1.39s/it]



 33%|███▎      | 104/313 [02:25<04:49,  1.39s/it]



 34%|███▎      | 105/313 [02:27<04:48,  1.39s/it]



 34%|███▍      | 106/313 [02:28<04:47,  1.39s/it]



 34%|███▍      | 107/313 [02:29<04:45,  1.39s/it]



 35%|███▍      | 108/313 [02:31<04:44,  1.39s/it]



 35%|███▍      | 109/313 [02:32<04:43,  1.39s/it]



 35%|███▌      | 110/313 [02:34<04:41,  1.39s/it]



 35%|███▌      | 111/313 [02:35<04:40,  1.39s/it]



 36%|███▌      | 112/313 [02:36<04:38,  1.39s/it]



 36%|███▌      | 113/313 [02:38<04:37,  1.39s/it]



 36%|███▋      | 114/313 [02:39<04:36,  1.39s/it]



 37%|███▋      | 115/313 [02:41<04:35,  1.39s/it]



 37%|███▋      | 116/313 [02:42<04:33,  1.39s/it]



 37%|███▋      | 117/313 [02:43<04:32,  1.39s/it]



 38%|███▊      | 118/313 [02:45<04:31,  1.39s/it]



 38%|███▊      | 119/313 [02:46<04:30,  1.

...

  Batch   300  of    313.    Elapsed: 0:06:59.






 96%|█████████▌| 301/313 [07:00<00:16,  1.40s/it]



 96%|█████████▋| 302/313 [07:01<00:15,  1.40s/it]



 97%|█████████▋| 303/313 [07:03<00:13,  1.39s/it]



 97%|█████████▋| 304/313 [07:04<00:12,  1.39s/it]



 97%|█████████▋| 305/313 [07:06<00:11,  1.40s/it]



 98%|█████████▊| 306/313 [07:07<00:09,  1.39s/it]



 98%|█████████▊| 307/313 [07:08<00:08,  1.40s/it]



 98%|█████████▊| 308/313 [07:10<00:06,  1.39s/it]



 99%|█████████▊| 309/313 [07:11<00:05,  1.40s/it]



 99%|█████████▉| 310/313 [07:13<00:04,  1.39s/it]



 99%|█████████▉| 311/313 [07:14<00:02,  1.40s/it]



100%|█████████▉| 312/313 [07:15<00:01,  1.39s/it]



100%|██████████| 313/313 [07:16<00:00,  1.40s/it]




  0%|          | 0/157 [00:00<?, ?it/s]


  Average training loss: 0.11
  Training epcoh took: 0:07:17

Running Validation...






  1%|          | 1/157 [00:00<01:17,  2.00it/s]



  1%|▏         | 2/157 [00:01<01:18,  1.98it/s]



  2%|▏         | 3/157 [00:01<01:17,  1.99it/s]



  3%|▎         | 4/157 [00:02<01:17,  1.97it/s]



  3%|▎         | 5/157 [00:02<01:17,  1.97it/s]



  4%|▍         | 6/157 [00:03<01:16,  1.96it/s]



  4%|▍         | 7/157 [00:03<01:16,  1.97it/s]



  5%|▌         | 8/157 [00:04<01:15,  1.96it/s]



  6%|▌         | 9/157 [00:04<01:15,  1.97it/s]



  6%|▋         | 10/157 [00:05<01:15,  1.96it/s]



  7%|▋         | 11/157 [00:05<01:14,  1.97it/s]



  8%|▊         | 12/157 [00:06<01:13,  1.96it/s]



  8%|▊         | 13/157 [00:06<01:12,  1.98it/s]



  9%|▉         | 14/157 [00:07<01:12,  1.97it/s]



 10%|▉         | 15/157 [00:07<01:11,  1.98it/s]



 10%|█         | 16/157 [00:08<01:11,  1.97it/s]



 11%|█         | 17/157 [00:08<01:10,  1.98it/s]



 11%|█▏        | 18/157 [00:09<01:10,  1.97it/s]



 12%|█▏        | 19/157 [00:09<01:09,  1.98it/s]



 13%|█▎        | 

  Accuracy: 27.51
  Validation Loss: 0.65
  Validation took: 0:01:19

Training complete!
Total training took 0:08:36 (h:mm:ss)
